In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import cv2
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src.datasets import get_dataset, get_dataloader
from src.utils import parse_data_cfg, IMG, FPHA, LMDB, DATA_DIR, YOLO

In [ ]:
exp_fold = 'exp0'
cfgname = 'yolov2_fpha_data'
cfg = parse_data_cfg(WORK_DIR/'data_cfg'/exp_fold/(cfgname + '.cfg'))
epoch = 300
exp_dir = cfg["exp_dir"]
data_split = 'test'
split_set = cfg[data_split + '_set']

# Visualize Dataloader

In [ ]:
dataset_kwargs = {'split_set': split_set}
dataset   = get_dataset(cfg, dataset_kwargs)
sampler   = None
shuffle   = cfg['shuffle']
kwargs = {'batch_size'  :   int(cfg['batch_size']),
          'shuffle'     :   shuffle,
          'num_workers' :   int(cfg['num_workers']),
          'pin_memory'  :   True}
data_loader = get_dataloader(dataset, sampler, kwargs)


In [ ]:
idx = 0 
for i, (img, bbox_gt) in enumerate(data_loader):
    if i == idx:
        batch_size = img.shape[0]
        img = img.cpu().numpy()
        img = np.swapaxes(img, 2, 3)
        img = np.swapaxes(img, 1, 3)
        img = IMG.scale_img_255(img)
        bbox_gt = bbox_gt.cpu().numpy()
        break
    i += 1

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(15, 15))
idx = 0
for i in range(4):
    for j in range(4):
        cur_img = img[idx]
        b = bbox_gt[idx]     
        ax[i, j].imshow(cur_img)
        FPHA.draw_bbox(ax[i, j], b, (cur_img.shape[1], cur_img.shape[0]))
        idx += 1

# Evaluation

In [ ]:
keys = LMDB.get_keys(os.path.join(DATA_DIR, split_set + "_keys_cache.p"))
bbox_gt = LMDB.read_all_lmdb_dataroot(keys, os.path.join(DATA_DIR, split_set + "_bbox_gt.lmdb"), "float32", 4)
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_bbox.txt'.format(epoch, data_split))
bbox_pred = np.loadtxt(pred_file)
bbox_pred = np.asarray([bbox[:5] for bbox in bbox_pred]) # take only the first bbox

In [ ]:
idx = 15430
img = np.asarray(Image.open(os.path.join(DATA_DIR, 'First_Person_Action_Benchmark', 'Video_files', keys[idx])))
print(keys[idx])
fig, ax = plt.subplots()
ax.imshow(img)
FPHA.draw_bbox(ax, bbox_pred[idx], (img.shape[1], img.shape[0]), 'r')
FPHA.draw_bbox(ax, bbox_gt[idx], (img.shape[1], img.shape[0]), 'b')

In [ ]:
iou_thresh = 0.5
correct = 0
avg_iou = 0
worst_iou = 10
worst_iou_idx = 0
iou_list = []
for i in range(len(bbox_pred)):
    iou = YOLO.bbox_iou(bbox_pred[i], bbox_gt[i])
    avg_iou += iou
    iou_list.append(iou)
    if iou < worst_iou:
        worst_iou = iou
        worst_iou_idx = i
    
    if iou > iou_thresh:
        correct += 1
        
recall = correct/len(bbox_pred)
avg_iou = avg_iou/len(bbox_pred)
print('Recall:', recall, 'Avg_IOU:', avg_iou)
print('Worst Bbox id:', worst_iou_idx)
for idx in np.argsort(iou_list):
    print(idx)

# Detect

In [ ]:
import torch

from src.models import get_model
from src.utils import EK

model = get_model(cfg, False, epoch, None, None)

In [ ]:
# Image

img = Image.open(EK.get_img_path(22, 16, 11616))
# idx = 1000
# img = Image.open(os.path.join(DATA_DIR, 'First_Person_Action_Benchmark', 'Video_files', keys[idx]))
box = model.detect(img)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
if(len(box.shape) == 2):
    for i, b in enumerate(box):
        c = 'r'
        if i == 1:
            c = 'b'
        FPHA.draw_bbox(ax, b, (img.size[0], img.size[1]), c)
else:
    FPHA.draw_bbox(ax, box, (img.size[0], img.size[1]))

In [ ]:
# Video

dom = 'action'
modality = 'rgb'
data_split_vid = 'train'

# Single vid
vid_idx = 53
all_img_path = EK.get_video_frames(vid_idx, dom=dom, modality=modality, data_split=data_split_vid)

# Multi vid
# start_vid = 54
# end_vid = 57
# vid_idx = '{}-{}'.format(start_vid, end_vid)
# all_img_path = []
# for idx in range(54, end_vid + 1):
#     cur_img_paths = EK.get_video_frames(idx, dom=dom, modality=modality, data_split=data_split_vid)
#     all_img_path += cur_img_paths

# print(len(all_img_path))

In [ ]:
from moviepy.editor import ImageSequenceClip
from tqdm import tqdm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import time
SAVE_DIR = Path(DATA_DIR)/'acv-data'/'gifs'

frames = []
total_time = 0
for img_path in tqdm(all_img_path):
    img = Image.open(img_path)
    start = time.time()
    box = model.detect(img)
    end = time.time()
    total_time += (end - start)
    fig, ax = plt.subplots()
    ax = fig.gca()
    ax.imshow(img)
    ax.axis('off')
    if(len(box.shape) == 2):
#         box = box[0]
        for i, b in enumerate(box):
            c = 'r'
            if i == 1:
                c = 'b'
            FPHA.draw_bbox(ax, b, (img.size[0], img.size[1]), c)
    else:
        FPHA.draw_bbox(ax, box, (img.size[0], img.size[1]))
    fig.canvas.draw()
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frames.append(data)
    plt.close()


print('fps:', len(all_img_path)/total_time)
segment_clip = ImageSequenceClip(frames, fps=60)
name = SAVE_DIR/('{}_{}_{}_{}_{}_{}.gif'.format(cfgname, epoch, vid_idx, dom, modality, data_split_vid))
segment_clip.write_gif(name, fps=6)
from IPython.display import Image as IPythonImage
with open(name,'rb') as f:
    display(IPythonImage(data=f.read(), format='png'))